# Query Translation Tools  

 > A library permits translation of complex boolean AND/OR queries between online APIs. 

In [ ]:
#| default_exp utils.langchain_utils
from nbdev import *

In [1]:
#| export
import json

from langchain.callbacks.base import BaseCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AgentAction, AgentFinish, HumanMessage, AIMessage
from langchain.schema import BaseMessage, BaseOutputParser, OutputParserException
from langchain.schema import LLMResult
from langchain.utils.input import print_text



import os
import re
import sys
from typing import (
    Any,
    Dict,
    List,
    Union,
    Optional
)

In [1]:
#| export

class JsonEnclosedByTextOutputParser(BaseOutputParser[Any]):
    """Parse the output of an LLM call to a JSON object."""

    def parse(self, text: str) -> Any:
        text = text.strip()
        m = re.search('.*([\[\{](.|\n)*[\}\]]).*', text, flags=re.M)
        if m:
            text1 = m.group(1)

            # need to make sure all entries in the JSON are quoted
            # so that the JSON parser can parse it
            # e.g. {"a": E} -> {"a": "E"}
            # this is a hack, but it works
            text2 = re.sub(r':\s*([a-zA-Z0-9_]+)\s*([,\]\}])', r': "\1"\2', text1)
            try:
                return json.loads(text2)
            except json.JSONDecodeError as e:
                raise OutputParserException(f"Invalid json output: {text2} derived from {text}") from e
        else: 
            raise OutputParserException(f"Could not find json-formatted data in: {text}")

    @property
    def _type(self) -> str:
        return "json_enclosed_by_text_output_parser"
    


NameError: name 'BaseOutputParser' is not defined

In [3]:
#| export

class suppress_stdout_stderr(object):
    def __enter__(self):
        self.outnull_file = open(os.devnull, 'w')
        self.errnull_file = open(os.devnull, 'w')

        self.old_stdout_fileno_undup    = sys.stdout.fileno()
        self.old_stderr_fileno_undup    = sys.stderr.fileno()

        self.old_stdout_fileno = os.dup ( sys.stdout.fileno() )
        self.old_stderr_fileno = os.dup ( sys.stderr.fileno() )

        self.old_stdout = sys.stdout
        self.old_stderr = sys.stderr

        os.dup2 ( self.outnull_file.fileno(), self.old_stdout_fileno_undup )
        os.dup2 ( self.errnull_file.fileno(), self.old_stderr_fileno_undup )

        sys.stdout = self.outnull_file        
        sys.stderr = self.errnull_file
        return self

    def __exit__(self, *_):        
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

        os.dup2 ( self.old_stdout_fileno, self.old_stdout_fileno_undup )
        os.dup2 ( self.old_stderr_fileno, self.old_stderr_fileno_undup )

        os.close ( self.old_stdout_fileno )
        os.close ( self.old_stderr_fileno )

        self.outnull_file.close()
        self.errnull_file.close()